# Advection-diffusion-reaction system used by Kopecz and Meister (2019)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp

import plot_fkt
plot_fkt.setup_plt()

import numpy.linalg as linalg

from numba import jit, float64, stencil

#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

from OrderCondition import *
from RKimple import *
import utils 

In [ ]:
def plot_stepsizes(status,slot = None):

    #At first plot the stepsizes according to the evalautions
    dt = np.array(status['dt_calc'])
    tol_met = np.array(status['sc'])
    neg = np.array(status['b'])
    n= np.arange(len(dt))
    plt.figure()
    plt.plot(n,dt,'-k')
    plt.plot(n[tol_met=='m'],dt[tol_met=='m'],'bx',label ='steps')
    plt.plot(n[tol_met!='m'],dt[tol_met!='m'],'rx',label ='tol not met')
    plt.plot(n[neg=='r'],dt[neg=='r'],'ro',label = 'infeasible')
    plt.xlim(slot)
    plt.grid()
    plt.legend()

    #plot error and change in commparison
    plt.figure()
    change = np.array(status['change'])
    error = np.array(status['error'])
    old_min = np.array(status['old_min'])
    plt.plot(change,'x',label = 'Change')
    plt.plot(error,'x',label = 'Error')
    plt.plot(old_min,'x' ,label = 'Min')
    plt.xlim(slot)
    #plt.ylim([-0,1])
    plt.grid()
    plt.legend()

In [ ]:
beta1 = (0.7/5)
beta2 = (0.3/5)
#compute a totla error out of error and change and use this as inmut for a single PI controller
def dt_logic_PI(stepsize_control,dt_old,dt_adp,error,change,success,tol_met):
    print('error:',error)

    facmax = 1.2
    facmin = 0.1
    fac = 10
    w_change = 1
    #compute total error
    if change == None:
        change = 10  #maybe set her some other value
    error_tot = error + w_change*change
    
    dt_old = dt_logic_PI.dt_old 

    #Control
    Tol = stepsize_control.a_tol
    dt = dt_old *(Tol/error_tot)**beta1*(dt_logic_PI.error_tot_old[-1]/error_tot)**beta2
    
    #Update storage vaiables
    dt_logic_PI.error_tot_old[:-1] = dt_logic_PI.error_tot_old[1:] 
    dt_logic_PI.error_tot_old[-1] = error_tot

    
    dt = min(facmax*dt_old,max(facmin*dt_old,dt))
    dt_logic_PI.dt_old=dt
    return max(dt,stepsize_control.dt_min)


dt_logic_PI.error_tot_old = np.zeros(3)
dt_logic_PI.dt_old=0.01

In [ ]:
# Now we implement the advection Diffusion equation
# To achieve compatibility with the existing time integrators the u's are stored in a single vector

#u= [u_1^T,u_2^T,u_3^T,u_4^T]^T

@stencil
def kernel_adv(a,dxi):
    return dxi*(a[0]-a[-1])

@stencil
def kernel_dif(a,dxqi):
    return dxqi*(a[-1]-2*a[0]+a[1])

@jit(float64[:](float64[:],float64),nopython=True)
def adv(u,dxi):
    du = kernel_adv(u,dxi)
    
    #Periodic
    du[0]=dxi*(u[0]-u[-1])
    return du

@jit(float64[:](float64[:],float64),nopython=True)
def dif(u,dxqi):
    du = kernel_dif(u,dxqi)

    #Periodic
    du[0] =dxqi*(u[-1]-2*u[0]+u[1])
    du[-1]=dxqi*(u[-2]-2*u[-1]+u[0])
    
    return du


@jit(float64[:](float64,float64[:]),nopython=True)
def f_ADP(t,u):
    a=1e-2
    d=1e-6
    
    du = np.zeros_like(u)
    
    #split u in different parts
    N = len(u)//4
    u1 = u[0:N]
    u2 = u[N:2*N]
    u3 = u[2*N:3*N]
    u4 = u[3*N:4*N]
    
    #calculate Production
    pu1 = 0.01*u2+0.01*u3+0.003*u4-(u1*u2)/(0.01+u1)
    pu2 = -0.01*u2-0.5*(1-np.exp(-1.21*u2**2))*u3-0.05*u2+(u1*u2)/(0.01+u1)
    pu3 = 0.5*(1-np.exp(-1.21*u2**2))*u3-0.01*u3-0.02*u3
    pu4 = 0.05*u2 + 0.02*u3-0.003*u4
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    
    du[0:N] = -a*adv(u1,1/dx)+d*dif(u1,1/dxq) + pu1 
    du[N:2*N] = -a*adv(u2,1/dx)+d*dif(u2,1/dxq) + pu2
    du[2*N:3*N] = -a*adv(u3,1/dx)+d*dif(u3,1/dxq) + pu3
    du[3*N:4*N] = -a*adv(u4,1/dx)+d*dif(u4,1/dxq) + pu4   
    
    return du


#Added a modification to the function that alters the behavior for values <0.005 to make shure that
#there is no second root of the stageeqation with negative us
@jit(float64[:](float64,float64[:]),nopython=True)
def f_ADP_pos(t,u):
    a=1e-2
    d=1e-6
    
    du = np.zeros_like(u)
    
    #split u in different parts
    N = len(u)//4
    u1 = np.where(u[0:N]>-0.005,u[0:N],-0.005)   #
    u2 = np.where(u[N:2*N]>-0.005,u[N:2*N],-0.005)
    u3 = np.where(u[2*N:3*N]>-0.005,u[2*N:3*N],-0.005)
    u4 = np.where(u[3*N:4*N]>-0.005,u[3*N:4*N],-0.005)
    
    
    
    #calculate Production
    pu1 = 0.01*u2+0.01*u3+0.003*u4-(u1*u2)/(0.01+u1)
    pu2 = -0.01*u2-0.5*(1-np.exp(-1.21*u2**2))*u3-0.05*u2+(u1*u2)/(0.01+u1)
    pu3 = 0.5*(1-np.exp(-1.21*u2**2))*u3-0.01*u3-0.02*u3
    pu4 = 0.05*u2 + 0.02*u3-0.003*u4
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    
    du[0:N] = -a*adv(u1,1/dx)+d*dif(u1,1/dxq) + pu1 
    du[N:2*N] = -a*adv(u2,1/dx)+d*dif(u2,1/dxq) + pu2
    du[2*N:3*N] = -a*adv(u3,1/dx)+d*dif(u3,1/dxq) + pu3
    du[3*N:4*N] = -a*adv(u4,1/dx)+d*dif(u4,1/dxq) + pu4   
    
    return du

In [ ]:
#Additional functions for better handling of System

def initial_ADP(N):
    start = np.array([8.,2.,1.,4.])
    u1s = np.zeros(N)
    u1s[0:N//2]=7
    u1s[N//2:]=8

    u2s = np.zeros(N)
    u2s[N//2:-1]=2

    u3s = np.zeros(N)
    u3s[N//2:]=1

    u4s = np.zeros(N)
    u4s[0:N//2]=1
    u4s[N//2:]=4


    u0=np.ones(N*4)
    u0[0:N] =  u1s
    u0[N:2*N] =  u2s
    u0[2*N:3*N] =  u3s
    u0[3*N:4*N] =  u4s
    
    return u0
    

def plot_image(u,t):
    N = u.shape[0]//4
    cmap=plt.get_cmap('plasma')
    display(u.shape)
    extent=[0,t[-1],1,0]
    plt.subplot(4, 1, 1)
    plt.imshow(u[0:N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 2)
    plt.imshow(u[N:2*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 3)
    plt.imshow(u[2*N:3*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 4)
    plt.imshow(u[3*N:4*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')

def plot_at_time(u,T,legend=True):
    n=np.argmin(np.abs(t-T))
    

    print(n,t[n])


    u_ = u[:,n]
    N= len(u_)//4

    u1 = u_[0:N]
    u2 = u_[N:2*N]
    u3 = u_[2*N:3*N]
    u4 = u_[3*N:4*N]

    x = np.linspace(0,1,101)
    plt.plot(x,np.append(u1,u1[:1]),label='$u_1$')
    plt.plot(x,np.append(u2,u2[:1]),label='$u_2$')
    plt.plot(x,np.append(u3,u3[:1]),label='$u_3$')
    plt.plot(x,np.append(u4,u4[:1]),label='$u_4$')
    plt.xlabel('$x$')
    plt.ylabel('$u_i$')
    plt.xlim([0,1])
    
    #plt.title('T='+str(T))
    if legend:
        plt.legend()

In [ ]:
#Reference Solution
"""
Idea: Make tolerance so tightly to make original solutions positive
"""
N =100
x = np.linspace(0,1,num = N)
dx=1/(N) # no -1 because of periodic boundary


u0 = initial_ADP(N)

ex3.bhat=np.array([-1,1,1,0,0,0])

dt_logic_PI.error_tot_old = np.zeros(3)
dt_logic_PI.dt_old=1e-9

solver_ex3 = Solver(rkm = ex3,
               dt = 1e-9,
               t_final = 60,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = 5,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               convex=False,
               solver_eqs=solver_nonlinear_arg,
               LP_opts = {'reduce':True,'verbose':False})

problem_ADR = Problem(f=f_ADP_pos,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

control = StepsizeControl(dt_min = 0,dt_max = np.infty,a_tol = 1e-15,r_tol=0.00001,f = dt_logic_PI
                          ,tol_reqect = 10*0.01)

beta1 = (0.7/5)
beta2 = (0.5/5)

status,t,u,b,KK = RK_integrate(solver=solver_ex3,problem=problem_ADR,stepsize_control=control,verbose=True,dumpK=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
N =100
x = np.linspace(0,1,num = N)
dx=1/(N) # no -1 because of periodic boundary


u0 = initial_ADP(N)

ex3.bhat=np.array([-1,1,1,0,0,0])

dt_logic_PI.error_tot_old = np.zeros(3)
dt_logic_PI.dt_old=0.01

solver_ex3 = Solver(rkm = ex3,
               dt = 0.01,
               t_final = 60,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = 5,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               convex=False,
               solver_eqs=solver_nonlinear_arg,
               LP_opts = {'reduce':True,'verbose':False})

problem_ADR = Problem(f=f_ADP_pos,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

control = StepsizeControl(dt_min = 0,dt_max = np.infty,a_tol = 0.01,r_tol=0.001,f = dt_logic_PI
                          ,tol_reqect = 10*0.01)

beta1 = (0.7/5)
beta2 = (0.5/5)

status,t,u,b,KK = RK_integrate(solver=solver_ex3,problem=problem_ADR,stepsize_control=control,verbose=True,dumpK=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
for T in (0,9,18,27,50):
    plt.figure(figsize=[6.4, 4])
    plot_at_time(u,T,legend=(T==0))
    plt.savefig('ADP_sol_'+"{:02d}".format(T)+'.pdf',bbox_inches = "tight")

In [ ]:
plot_stepsizes(status)

In [ ]:
rkm = ex3
plt.figure(figsize=[6.4*2, 4.7*2])
dt = np.array(status['dt_calc'])
old_min = np.array(status['old_min'])
new_min = np.array(status['new_min'])

change = np.array(status['change'])
error = np.array(status['error'])    

plt.subplot(5,1,1)
plt.plot(t,dt,'xC1')
plt.ylabel('$\Delta t$')
plt.grid()
plt.xlim([-1,61])
plt.ylim([-0.2,1.8])
plt.gca().tick_params(labelbottom=False)    


plt.subplot(5,1,2)
plt.plot(t,old_min,'x',label='before adaptation')
plt.plot(t,new_min,'+',label='after adaptation')
plt.grid()
plt.xlim([-1,61])
ax = plt.gca()
ax.set_yticks((0,-0.0025,-0.0050))
ax.set_yticklabels(('$0$', r'$-2.5 \times 10^{-3}$', r'$-5 \times 10^{-3}$'))
plt.legend()
plt.ylabel(r'$\min(u)$')
plt.gca().tick_params(labelbottom=False)    


plt.subplot(5,1,3)
plt.plot(t,error,'x',label = '$err_{T}$')
plt.plot(t,change,'+',label = r'$\delta$')
plt.xlim([-1,61])
plt.grid()
plt.legend()
plt.ylabel('Error')
plt.gca().tick_params(labelbottom=False)    



plt.subplot(5,1,4)
b_orig = rkm.b.copy()
b_orig.shape=(len(b_orig),1)
plt.plot(t[1:],np.linalg.norm((b-b_orig),axis=0,ord=1)[1:],'xC1')
plt.xlim([-1,61])
#plt.ylim([-2.5,2])
plt.ylabel(r'$\| \tilde{b} - b \|_1 $')
plt.grid()
plt.gca().tick_params(labelbottom=False)    


plt.subplot(5,1,5)
mass = np.sum(u,axis=0)
plt.plot(t,mass-mass[0],'xC1')
plt.ylabel(r'$\sum u - \sum u(0)$')#
ax = plt.gca()
ax.set_yticks((0,-2.5e-13,-5e-13))
ax.set_yticklabels(('$0$', r'$-2.5 \times 10^{-13}$',r'$-5 \times 10^{-13}$'))
plt.ylim([-7e-13,2e-13])
plt.xlim([-1,61])
plt.grid()

plt.xlabel('$t$')

plt.savefig('ADP_stepsize_b.pdf',bbox_inches = "tight")

In [ ]:
#Stats:
bs=np.array(status['b'])
print('Number of steps:',np.sum(bs!=None))
print('Number of adaptions:',np.sum(bs=='c'))
np.min(u)

In [ ]:
#check linear invariants:
mass = np.sum(u,axis=0)

In [ ]:
plt.plot(t,mass-mass[0])

In [ ]:
mass[1]

In [ ]:
print(utils.get_max_iter_h(status))